In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from visualize import PyramidVisualizer, get_dataset, simple_visualizer
from metroem.aligner import Aligner

In [35]:
import modelhouse
import json

model_path = "matrix://corgie-models/FAFB_align_m5"
       
aligner = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
      "checkpoint_name": "test_no_batchnorm",
      "finetune": True,
      "finetune_iter": 200,
      "finetune_lr": 0.3,
      "finetune_sm": 300,
      "sm_defect_coarsening": [[1, 0.0], [20, 0.3], [40, 0.6]],
      "crop": 32
    })
)

/usr/people/popovych/.modelhouse/tmp_files/tmp0xbwy941/model/model_spec.json
block_6convs_fms8to32_skip25.json
categorical/categorical_compch15_gridfalse_maxvalue7_outch2_stepnull_trainsttrue.json
block_6convs_fms8to32_skip25.json
categorical/categorical_compch15_gridfalse_maxvalue7_outch2_stepnull_trainsttrue.json
identity.json
embedder_m0_3x3_fms3x.json
block_5convs_7x7_fms1to3_skip14.json
block_3convs_3x3_fms3to32to3.json
average_pool.json
embedder_fms4to3.json
block_3convs_3x3_fms4to32to3.json
average_pool.json
block_3convs_3x3_fms3to32to3.json
identity.json
identity.json
identity.json
upsample_residuals.json
identity.json


/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/artificery/parsers/categorical_regression/parse.py:38: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  requires_grad=False) - self.component_channels//2


In [26]:
import cloudvolume as cv
import torch

raw_img_path = 'matrix://seamless-paper/prod_x3/normed/img/img_norm'


#z = 226
z = 226
#x_start = 140000
#y_start = 60000

x_start = 130000
y_start = 50000
img_mip = 5

x_size = 1024 * 2**img_mip
y_size = x_size
pad = 64 * 2**3

img_cv = cv.CloudVolume(raw_img_path, mip=img_mip, parallel=True, progress=False)
img1_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z:z+1].squeeze()
img2_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z+1:z+2].squeeze()

img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)

src_tgt_t = torch.cat([img1_t, img2_t], 1)
#pred_res = aligner(src_img=img1_t, tgt_img=img2_t)

In [ ]:
'''MIP5 with net


0.5840146541595459secs for net
2.9854114 2.8923268 0.09308447
New best: 92, No impr: 0, Iter: 199
2.2851658 2.0513072 0.23385859
12.488795042037964
'''

'''MIP5 without net


0.6791865825653076secs for net
3.17221 3.17221 0.0
New best: 117, No impr: 0, Iter: 199
2.2737517 2.0241246 0.24962708
12.609593868255615
'''

In [36]:
img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)

src_tgt_t = torch.cat([img1_t, img2_t], 1)
pred_res = aligner(src_img=img1_t, tgt_img=img2_t)
pred_res[:, :, 20:-20, 20:-20].abs().max()

0.6760382652282715secs for net
3.17221 3.17221 0.0
New best: 102, No impr: 0, Iter: 199
2.423286 2.2200677 0.20321824
12.755868434906006


tensor(7.4958, device='cuda:0', grad_fn=<MaxBackward1>)

In [37]:
simple_visualizer().visualize([pred_res, pred_res.from_pixels()(img1_t), img1_t, img2_t])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3), value=0), IntText(value=1, des…

In [6]:
import random
import torch
import numpy as np

import cloudvolume as cv
import torch

from visualize import PyramidVisualizer, get_dataset, simple_visualizer
from metroem.aligner import Aligner

#raw_img_path = 'gs://corgie_package/paper/basil_data_x0/aligned_x0/img/img_full_x0_rend'
raw_img_path = 'https://s3-hpcrc.rc.princeton.edu/fafbv15-em/v1'
raw_img_path = 'matrix://seamless-paper/prod_x3/normed/img/img_norm'


img_mip = 3

z = 500
x_start = 115000
y_start = 50000

x_size = 1024 * 2**img_mip
y_size = x_size
pad = 64 * 2**3
#
#img_cv = cv.CloudVolume(raw_img_path, mip=img_mip, parallel=True, progress=False)
#masked_img_cv = cv.CloudVolume(masked_img_path, mip=masked_img_mip, parallel=True, progress=False)
#field_cv = cv.CloudVolume(field_path, mip=field_mip, parallel=True, progress=False)
img_cv = cv.CloudVolume(raw_img_path, mip=img_mip, parallel=True, progress=False)
img1_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z:z+1].squeeze()
img2_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z+1:z+2].squeeze()

In [13]:
f"X: {(x_start-pad)//2**img_mip}-{(x_start+x_size+pad)//2**img_mip}" + f"  Y: {(y_start-pad)//2**img_mip}-{(y_start+y_size+pad)//2**img_mip}"

'X: 14311-15463  Y: 6186-7338'

In [15]:
img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)

src_tgt_t = torch.cat([img1_t, img2_t], 1)
pred_res = aligner(src_img=img1_t, tgt_img=img2_t)

In [16]:
img1_t.mean()

tensor(0.1529, device='cuda:0')

In [17]:
pred_res.max()

tensor(6.0104, device='cuda:0')

In [18]:
simple_visualizer().visualize([pred_res, pred_res.from_pixels()(img1_t), img1_t, img2_t])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3), value=0), IntText(value=1, des…

In [19]:

simple_visualizer().visualize([pred_res, pred_res.from_pixels()(img1_t), img1_t, img2_t])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3), value=0), IntText(value=1, des…

In [20]:
import metroem
from metroem.helpers import get_np
mask = (img1_np != 0).astype(np.float32)
closed = metroem.masks.closing(mask, n=40).astype(np.float32)
closed_2 = ndimage.binary_closing(mask, iterations=40).astype(np.int)
simple_visualizer().visualize([get_np(src_defects).astype(np.float32), get_np(src_zeros).astype(np.float32), (mask == 0) * closed, closed, closed_2, img1_np, mask, closed])

NameError: name 'ndimage' is not defined

In [ ]:
ndimage.binary_closing(a).astype(np.int)

In [19]:
a = np.zeros((5,5), dtype=np.int)

/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [20]:
a[1:-1, 1:-1] = 1; a[2,2] = 0

In [21]:
a

array([[0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]])

In [22]:
simple_visualizer().visualize([a])

interactive(children=(ToggleButtons(description='Image:', options=(0,), value=0), IntText(value=1, description…

NameError: name 'ndimage' is not defined

In [24]:
from scipy import ndimage

In [26]:
once = ndimage.binary_closing(a).astype(np.int)

/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [27]:
simple_visualizer().visualize([once])

interactive(children=(ToggleButtons(description='Image:', options=(0,), value=0), IntText(value=1, description…